### Singular Value Decomposition (SVD) and Randomized SVD

The SVD algorithm factorizes a matrix into one matrix with orthogonal columns and one with orthogonal rows (along with a diagonal matrix, which contains the relative importance of each factor).

SVD is excellent, but it can be very slow. To save time, we can use another method called Randomized SVD which performs SVD on a smaller matrix that has approximately the same range as our original matrix and gets an acceptable result much faster.

Below are some examples of applying SVD and randomized SVD on topic modeling.

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline
np.set_printoptions(suppress=True)

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, remove=remove)

In [3]:
# Create matrix from data

vectorizer = CountVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(newsgroups_train.data).todense() # (documents, vocab)
vectors.shape #, vectors.nnz / vectors.shape[0], row_means.shape

(2034, 26576)

In [4]:
vocab = np.array(vectorizer.get_feature_names())

In [5]:
num_top_words=8

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = [top_words(t) for t in a]
    return [' '.join(t) for t in topic_words]

### full SVD

In [6]:
from scipy import linalg

%time U, s, Vh = linalg.svd(vectors, full_matrices=False)

CPU times: user 52.7 s, sys: 1.56 s, total: 54.3 s
Wall time: 15.4 s


In [7]:
show_topics(Vh[:10])

['ditto critus propagandist surname galacticentric kindergarten surreal imaginative',
 'jpeg gif file color quality image jfif format',
 'graphics edu pub mail 128 3d ray ftp',
 'jesus god matthew people atheists atheism does graphics',
 'image data processing analysis software available tools display',
 'god atheists atheism religious believe religion argument true',
 'space nasa lunar mars probe moon missions probes',
 'image probe surface lunar mars probes moon orbit',
 'argument fallacy conclusion example true ad argumentum premises',
 'space larson image theory universe physical nasa material']

### randomized SVD

##### 1. sklearn

In [8]:
from sklearn import decomposition

%time U, s, Vh = decomposition.randomized_svd(vectors, 10)

CPU times: user 14.1 s, sys: 3.05 s, total: 17.1 s
Wall time: 5.56 s


In [9]:
show_topics(Vh)

['jpeg image edu file graphics images gif data',
 'jpeg gif file color quality image jfif format',
 'space jesus launch god people satellite matthew atheists',
 'jesus god matthew people atheists atheism does graphics',
 'image data processing analysis software available tools display',
 'jesus matthew prophecy messiah psalm isaiah david said',
 'launch commercial satellite market image services satellites launches',
 'data available nasa ftp grass anonymous contact gov',
 'argument fallacy conclusion example true ad argumentum premises',
 'probe data surface moon mars probes lunar launch']

##### 2. fbpca

In [10]:
import fbpca

%time  U, s, Vh = fbpca.pca(vectors, 10)

CPU times: user 5.24 s, sys: 1.17 s, total: 6.41 s
Wall time: 2.07 s


In [11]:
show_topics(Vh)

['kent cheers bobby p3 islamic p2 den p1',
 'jpeg gif file color quality image jfif bit',
 'graphics edu pub mail 128 3d ray send',
 'jesus god matthew people atheists atheism does said',
 'image data processing analysis software available tools tool',
 'god atheists atheism religious believe argument religion true',
 'space larson nasa theory universe physical star lunar',
 'nasa available data ftp god space gov ra',
 'atheists religious atheism image god religion larson probe',
 'space image nasa edu sci news processing shuttle']

##### 3. simple implement

In [12]:
def simple_randomized_svd(M, k=10):
    m, n = M.shape
    transpose = False
    if m < n:
        transpose = True
        M = M.T
        
    rand_matrix = np.random.normal(size=(M.shape[1], k))  # short side by k
    Q, _ = np.linalg.qr(M @ rand_matrix, mode='reduced')  # long side by k
    smaller_matrix = Q.T @ M                              # k by short side
    U_hat, s, V = np.linalg.svd(smaller_matrix, full_matrices=False)
    U = Q @ U_hat
    
    if transpose:
        return V.T, s.T, U.T
    else:
        return U, s, V

In [13]:
%time  U, s, Vh = simple_randomized_svd(vectors, 10)

CPU times: user 1.65 s, sys: 384 ms, total: 2.04 s
Wall time: 687 ms


In [14]:
show_topics(Vh.tolist())

['argument fallacy conclusion op_cols op_rows int col row',
 'edu graphics pub data ftp 3d ray package',
 'graphics pub edu ray argument jpeg god mail',
 'jesus atheists god atheism religion matthew religious people',
 'image god data believe larson universe theory think',
 'god ra den p3 p2 atheists satellite p1',
 'argument fallacy example conclusion p2 p3 den used',
 'jesus science image launch graphics gay used april',
 'image want good ra program memory read john',
 'ra center tyre ezekiel books read atheism xv']

##### 4.  improved version

In [15]:
# computes an orthonormal matrix whose range approximates the range of M
# power_iteration_normalizer can be safe_sparse_dot (fast but unstable), LU (in between), or QR (slow but most accurate)
def randomized_range_finder(M, k, n_iter=10):
    Q = np.random.normal(size=(M.shape[1], k))
    
    for i in range(n_iter):
        Q, _ = linalg.lu(M @ Q, permute_l=True)
        Q, _ = linalg.lu(M.T @ Q, permute_l=True)
        
    Q, _ = linalg.qr(M @ Q, mode='economic')
    return Q

In [16]:
def randomized_svd(M, k=10, n_oversamples=10, n_iter=10):
    k += n_oversamples
    
    m, n = M.shape
    transpose = False
    if m < n:
        transpose = True
        M = M.T
        
    Q = randomized_range_finder(M, k, n_iter)
    
    smaller_matrix = Q.T @ M                              # k by short side
    U_hat, s, V = np.linalg.svd(smaller_matrix, full_matrices=False)
    U = Q @ U_hat
    
    k -= n_oversamples
    
    if transpose:
        return V.T[:, :k], s.T[:k], U.T[:k, :]
    else:
        return U[:, :k], s[:k], V[:k, :]

In [17]:
%time  U, s, Vh = simple_randomized_svd(vectors, 10)

CPU times: user 1.54 s, sys: 381 ms, total: 1.92 s
Wall time: 685 ms


In [18]:
show_topics(Vh.tolist())

['launch lunar satellite orbit shuttle objective market flight',
 'jpeg gif file color quality bit argument format',
 'edu image graphics law pub send 3d stuff',
 'jesus image people data atheism available atheists god',
 'atheists religious god atheism don space religion shuttle',
 'image life launch processing images tools analysis does',
 'larson space theory list email physical data general',
 'atheists moon image atheism true used larson fallacy',
 'god does program know satellite earth telescope change',
 'van software god edu het nasa uci dr']